# Can GPT-3 Pass the Canadian Citizenship Test?

9 January 2023

**Sean Rehaag**\
Director, Centre for Refugee Studies\
Director, Refugee Law Lab\
Associate Professor, Osgoode Hall Law School\
York University

One of the requirements to obtain Canadian citizenship is to pass a multiple-choice test that that involves questions about Canadian history, geography, economy, government, laws, and important symbols.

Details about the test, along with a study guide, are available [here](https://www.canada.ca/en/immigration-refugees-citizenship/services/canadian-citizenship/become-canadian-citizen/citizenship-test.html).

The notebook, *Can_gpt_pass_citizenship_test.ipynb*, examines whether, as of January 2023, OpenAI's GPT-3 can pass the citizenship test without any fine-tuning, using 60 practice questions made available by the [Toronto Public Library](https://www.torontopubliclibrary.ca/new-to-canada/citizenship.jsp).

You can scrape practice questions and answers from the Toronto Public Library's website by running the *ScrapeCitizenshipTest.py* file from a terminal (scraping uses playwright, which does not work well in Jupyter Notebook):

>pip install pandas
>
>pip install playwright
>
>playwright install
>
>python -m ScrapeCitizenshipTest

Alternatively, you can run the notebook using the Excel file with the scraped data in this repo.

Requirements for the notebook:

>pip install pandas
> 
>pip install openai

To run the notebook, you will also need an API key from OpenAI. Details about obtaining an account are [here](https://beta.openai.com/signup), and details about getting an API key are [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key). Once you have an API key, place it in a text file called SECRETS-OPENAI.txt in the project secrets folder using the template in the folder.

At the end of the notebook, a no-code method to produce similar results using ChatGPT is outlined.

To be cited as: Sean Rehaag, "Can GPT-3 Pass the Canadian Citizenship Test?" (9 January 2022), online: \<https://github.com/Refugee-Law-Lab/gpt3-canadian-citizenship-test\>.

License: [CC BY-NC-SA/4.0](https://creativecommons.org/licenses/by-nc-sa/4.0)

### Setup

In [1]:
import pandas as pd
import openai
import time

with open('secrets/SECRETS-OPENAI.txt', 'r') as in_file:
    openai.api_key = in_file.readlines()[0]

In [2]:
# load practice questions
# source: https://www.torontopubliclibrary.ca\new-to-canada/citizenship.jsp
# Scraped using: ScrapeCitizenshipTest.py

df = pd.read_excel('questions.xlsx')

# get letter for correct answers
def get_correct_letter(x):
    if x['correct_answer'] in x['answerA']:
        return 'a'
    elif x['correct_answer'] in x['answerB']:
        return 'b'
    elif x['correct_answer'] in x['answerC']:
        return 'c'
    elif x['correct_answer'] in x['answerD']:
        return 'd'
    else:
        return ''

df['correct_letter'] = df.apply(get_correct_letter, axis=1)

df

,question,answerA,answerB,answerC,answerD,correct_answer,correct_letter
0,Name two fundamental freedoms under Canadian law.,Equality rights and care for the environment,Aboriginal rights and conserving water,Freedom of speech and freedom of religion,The Magna Carta and English common law,Freedom of speech and freedom of religion,c
1,List four additional rights Canadian citizens ...,The right to be educated in either official la...,The right of be educated in either official la...,The right to enter and leave Canada and the Un...,"The right to live and work anywhere in Canada,...",The right to be educated in either official la...,a
2,Name three responsibilities of Canadian citize...,"Serving on a jury, keeping your yard tidy and ...","Obeying the law, voting in elections and worki...","Obeying the law, voting in elections and takin...","Voting in elections, taking responsibility for...","Obeying the law, voting in elections and takin...",c
3,Give an example of how you can help in the com...,Wear red on Canada Day,Drive to work,Volunteer at a food bank,Wash your car,Volunteer at a food bank,c
4,"What is meant by the ""equality of women and men""?",Men and women are treated equally under the law.,Men and women are the same,Men and women are similar,Men and women need to obey the law,Men and women are treated equally under the law.,a
5,Who has the right to vote in federal elections?,Canadian citizens who are at least 19 years of...,Canadian citizens who are at least 18 years of...,Canadian citizens and landed immigrants who ar...,Canadian citizens and landed immigrants who ar...,Canadian citizens who are at least 18 years of...,b
6,List three ways you can protect the environment.,"Recycle, conserve energy and water and join a ...","Recycle, conserve energy and drive your car to...","Recycle, conserve energy and mow your lawn","Recycle, conserve energy and buy bottled water","Recycle, conserve energy and water and join a ...",a
7,"What does ""due process"" mean?",You must prove your innocence in court,The government must respect all the legal righ...,The government must respect most of the legal ...,"The law applies to everyone in Canada, except ...",The government must respect all the legal righ...,b
8,What do you promise when you take the oath of ...,"To pledge your loyalty to the Sovereign, His M...",To pledge your allegiance to the flag and fulf...,To pledge your allegiance to the Canadian Cons...,To pledge your loyalty to Canada from sea to sea,"To pledge your loyalty to the Sovereign, His M...",a
9,Name the three levels of government in Canada,"Federal, state and municipal","Federal, provincial or territorial and state","Federal, provincial or territorial and municipal","Federal, provincial or territorial and local","Federal, provincial or territorial and municipal",c


### Submit practice questions to GPT-3

The code uses a few shot learning approach. It provides GPT-3 with a prompt containing contextual information (i.e. that the system is being asked to answer questions from a Canadian citizenship test), with an example of a question with an answer in the format sought, and with the actual practice question and possible answers. The system returns the letter corresponding to the answer that it thinks best responds to the question.

In [3]:
# Get answer from GPT-3

def get_completion(prompt, model_to_use):
    for attempt in range (5):
        try:
            model_output = openai.Completion.create(
            model=model_to_use,
            prompt=prompt,
            temperature= 0,
            max_tokens = 4,
            stop='\n')
            return model_output
        except:
            time.sleep(5)

def get_answer(x):
        
    header = 'Please answer this question from a Canadian citizenship test, providing only the letter of the answer that is most likely correct. \n\n'
    header_question = 'Question 1: Why is the battle of Vimy Ridge important to Canadians?\n'
    header_answers = []
    header_answers.append('Possible answers:\n')
    header_answers.append(' a: It was the last battle of the First World War\n')
    header_answers.append(' b: It was an important victory in the Boer War\n')
    header_answers.append(' c: It has come to symbolize Canada\'s coming of age as a nation\n')
    header_answers.append(' d: Out of it was formed the Canadian Corps\n\n')
    header_answers = ''.join(header_answers)
    header_completion = 'Selected answer: c \n\n'

    question = 'Question 2: ' + x['question'] + '\n'
    answers = []
    answers.append('Possible answers: \n')
    answers.append(' a: ' + x['answerA'] + '\n')
    answers.append(' b: ' + x['answerB'] + '\n')
    answers.append(' c: ' + x['answerC'] + '\n')
    answers.append(' d: ' + x['answerD'] + '\n\n')
    answers = ''.join(answers)
    completion_prompt = 'Selected answer:'

    prompt = header + header_question + header_answers + header_completion + question + answers + completion_prompt

    model_to_use = 'text-davinci-003'
    
    completion = get_completion(prompt, model_to_use)

    return completion['choices'][0]['text'].strip()

df['gpt_answer'] = df.apply(get_answer, axis=1)

df['gpt_correct']=df.apply(lambda x: True if x['gpt_answer']==x['correct_letter'] else False, axis=1)

print('Percentage of GPT-3 answers that are correct: ', 100 * len(df[df['gpt_correct']==True])/len(df))

df

Percentage of GPT-3 answers that are correct:  88.33333333333333


,question,answerA,answerB,answerC,answerD,correct_answer,correct_letter,gpt_answer,gpt_correct
0,Name two fundamental freedoms under Canadian law.,Equality rights and care for the environment,Aboriginal rights and conserving water,Freedom of speech and freedom of religion,The Magna Carta and English common law,Freedom of speech and freedom of religion,c,c,True
1,List four additional rights Canadian citizens ...,The right to be educated in either official la...,The right of be educated in either official la...,The right to enter and leave Canada and the Un...,"The right to live and work anywhere in Canada,...",The right to be educated in either official la...,a,d,False
2,Name three responsibilities of Canadian citize...,"Serving on a jury, keeping your yard tidy and ...","Obeying the law, voting in elections and worki...","Obeying the law, voting in elections and takin...","Voting in elections, taking responsibility for...","Obeying the law, voting in elections and takin...",c,c,True
3,Give an example of how you can help in the com...,Wear red on Canada Day,Drive to work,Volunteer at a food bank,Wash your car,Volunteer at a food bank,c,c,True
4,"What is meant by the ""equality of women and men""?",Men and women are treated equally under the law.,Men and women are the same,Men and women are similar,Men and women need to obey the law,Men and women are treated equally under the law.,a,a,True
5,Who has the right to vote in federal elections?,Canadian citizens who are at least 19 years of...,Canadian citizens who are at least 18 years of...,Canadian citizens and landed immigrants who ar...,Canadian citizens and landed immigrants who ar...,Canadian citizens who are at least 18 years of...,b,b,True
6,List three ways you can protect the environment.,"Recycle, conserve energy and water and join a ...","Recycle, conserve energy and drive your car to...","Recycle, conserve energy and mow your lawn","Recycle, conserve energy and buy bottled water","Recycle, conserve energy and water and join a ...",a,a,True
7,"What does ""due process"" mean?",You must prove your innocence in court,The government must respect all the legal righ...,The government must respect most of the legal ...,"The law applies to everyone in Canada, except ...",The government must respect all the legal righ...,b,b,True
8,What do you promise when you take the oath of ...,"To pledge your loyalty to the Sovereign, His M...",To pledge your allegiance to the flag and fulf...,To pledge your allegiance to the Canadian Cons...,To pledge your loyalty to Canada from sea to sea,"To pledge your loyalty to the Sovereign, His M...",a,b,False
9,Name the three levels of government in Canada,"Federal, state and municipal","Federal, provincial or territorial and state","Federal, provincial or territorial and municipal","Federal, provincial or territorial and local","Federal, provincial or territorial and municipal",c,c,True


In [4]:
# Export to dataframe excel
df.to_excel('questions_with_gpt_answers.xlsx', index = False)

# So, can GPT-3 pass the Canadian citizenship test?

Yes -- and quite easily. The passing grade is 75% (on 20 questions), while GPT-3 typically gets between 88% and 92%. As a comparison, in 2019 a [poll](https://www.ctvnews.ca/canada/nearly-9-in-10-canadians-would-fail-the-citizenship-test-poll-1.4489704) found that 9 out of 10 Canadian citizens would not pass the test.

Overall, GPT-3 did very well on the citizenship test. Some of the questions that GPT-3 struggled with may relate to outdated training data, but for other questions it is not clear why GPT-3 had difficulties. 

On a recent run, out of 60 questions, GPT-3 got these answers wrong:

#### List four additional rights Canadian citizens have under the Canadian Charter of Rights and Freedoms.
  a: The right to be educated in either official language, the right to apply for a Canadian passport, the right to vote, the right to enter and leave Canada freely	
  
  b: The right of be educated in either official language, the right to apply for a Canadian passport, the right to drive, the right to enter and leave Canada freely	
  
  c: The right to enter and leave Canada and the United States freely, the right to apply for a Canadian passport, the right to be educated, the right to vote	
  
  d: The right to live and work anywhere in Canada, the right to vote, the right to be educated in either official language, the right to drive

The answer is (d) but GPT-3 selects (a).

#### What do you promise when you take the oath of citizenship?
  a: To pledge your loyalty to the Sovereign, His Majesty King Charles III

  b: To pledge your allegiance to the flag and fulfill the duties of a Canadian

  c: To pledge your allegiance to the Canadian Constitution
  
  d: To pledge your loyalty to Canada from sea to sea

The answer is (a) but GPT-3 selects (b). This may be because GPT-3's training data predates the death of Queen Elizabeth II.

#### What are the three parts of Parliament?
  a: The Sovereign, the House of Commons and the Senate

  b: The Sovereign, the Prime Minister and the Premiers

  c: The Sovereign, the Governor General and the Prime Minister

  d: The Prime Minister, the House of Commons and the Senate

The answer is (a) but GPT-3 selects (d).

#### How often does an election have to be held according to the Constitution?
  a: Within three years of the last election

  b: Every four years following the most recent general election

  c: Within five years of the last election

  d: Whenever the Sovereign decides

The answer is (b) but GPT-3 selects (a).

#### Name the federal political parties represented in the House of Commons and their leaders
  a: Conservative Party of Canada (Pierre Poilievre), New Democratic Party (Jagmeet Singh), Liberal Party of Canada (Justin Trudeau), Bloc Québécois (Yves-François Blanchet), Green Party (Elizabeth May and Mike Morrice)

  b: Conservative Party of Canada (Pierre Poilievre), Liberal Party of Canada (Justin Trudeau), New Democratic Party (Jagmeet Singh), Bloc Québécois (Yves-François Blanchet)

  c: Conservative Party of Canada (Pierre Poilievre), Liberal Party of Canada (Justin Trudeau), Green Party (Elizabeth May and Mike Morrice),  Bloc Québécois (Yves-François Blanchet)

  d: Conservative Party of Canada (Pierre Poilievre), Liberal Party of Canada (Justin Trudeau), New Democratic Party (Jagmeet Singh), Green Party (Elizabeth May and Mike Morrice)

The answer is (a), but GPT-3 selects (d). It is not surprising that GPT-3 struggled with this question, as none of these answers were correct at the time of this model's version of GPT-3 training data (June 2021).

#### Name two responsibilities of the provincial government.

  a: Communication and education

  b: Health and recycling

  c: Education and health

  d: Education and defence

The answer is (c) but GPT-3 selects (b).

#### What are the first two lines of the National Anthem, O Canada?	

  a: "O Canada! Our home and native land! / With glowing hearts we see thee rise"	

  b: "O Canada! Our home and native land! / True patriot love in all of us command"

  c: "O Canada! Our home and native land! / God keep our land glorious and free"

  d: "God keep our land glorious and free / O Canada, we stand on guard for thee."

The answer is (b), but GPT-3 selects (a).

# A no-code alternative approach using ChatGPT

It is also possible to use OpenAI's GPT to answer questions about the
Canadian citizenship test without any coding.

This can be done using ChatGPT.

Go to https://chat.openai.com/chat.

Then, try pasting this prompt into ChatGPT:

> Please answer this question from a Canadian citizenship test, providing only the letter of the answer that is most likely correct.
>
>Example Question: Why is the battle of Vimy Ridge important to Canadians?
>
>Possible answers: 
>
> a: It was the last battle of the First World War
>
> b: It was an important victory in the Boer War 
>
> c: It has come to symbolize Canada\'s coming of age as a nation 
>
> d: Out of it was formed the Canadian Corps 
>
>
>Selected answer: c
>
>Question: Name two fundamental freedoms under Canadian law.
>
>Possible answers:
>
> a: Equality rights and care for the environment
>
> b: Aboriginal rights and conserving water
>
> c: Freedom of speech and freedom of religion
>
> d: The Magna Carta and English common law
>
> Selected answer:

ChatGPT is likely to return 'c', which is the correct answer.

Try again by pasting a second prompt into Chat GPT:

>Please answer this question from a Canadian citizenship test, providing only the letter of the answer that is most likely correct.
>
>Example Question: Why is the battle of Vimy Ridge important to Canadians?
>
>Possible answers: 
>
> a: It was the last battle of the First World War
>
> b: It was an important victory in the Boer War 
>
> c: It has come to symbolize Canada\'s coming of age as a nation 
>
> d: Out of it was formed the Canadian Corps 
>
>Selected answer: c
>
>Question: What is meant by a "secret ballot"?
>
> a: An unknown voting location
>
> b: A private candidates' meeting
>
> c: No one can accompany you to a polling station
>
> d: No one can watch you vote and no one should look at how you voted
>
>Selected answer:

ChatGPT is likely to return d, which is the correct answer.

In general ChatGPT should have slightly higher scores than GPT-3 because of how it was trained.